In [2]:
import edgar
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bsoup
import numpy as np
from collections import defaultdict

The following blocks of code pulls MD&A sections from companies that file the MD&A disclosures in a seperate exhibit 

In [179]:
IBMyears = [i for i in range(2009,2013)]
IBMresLst = ['https://www.sec.gov/Archives/edgar/data/51143/000104746910001151/a2195966zex-13.htm','https://www.sec.gov/Archives/edgar/data/51143/000104746911001117/a2201254zex-13.htm','https://www.sec.gov/Archives/edgar/data/51143/000104746912001742/a2206744zex-13.htm','https://www.sec.gov/Archives/edgar/data/51143/000104746913001698/a2212340zex-13.htm']

WFyears = [i for i in range(2013,2016)]
WFresLst = ['https://www.sec.gov/Archives/edgar/data/72971/000007297114000337/wfc10k_20131231-ex13.htm','https://www.sec.gov/Archives/edgar/data/72971/000007297115000449/wfc-12312014xex13.htm','https://www.sec.gov/Archives/edgar/data/72971/000007297115000449/wfc-12312014xex13.htm']

JJyears = [i for i in range(2007,2015)]
JJyears.remove(2011)
JJyears.remove(2010)
JJresLst = ['https://www.sec.gov/Archives/edgar/data/200406/000095012308002130/y47910exv13.htm','https://www.sec.gov/Archives/edgar/data/200406/000095012309003187/y74152exv13.htm','https://www.sec.gov/Archives/edgar/data/200406/000095012310019392/y80744exv13.htm','https://www.sec.gov/Archives/edgar/data/200406/000020040615000004/ex13-form10xk20141228.htm']

WMyears = [i for i in range(2007,2015)]
WMresLst = ['https://www.sec.gov/Archives/edgar/data/104169/000119312508071085/dex13.htm','https://www.sec.gov/Archives/edgar/data/104169/000010416909000006/dex13.htm','https://www.sec.gov/Archives/edgar/data/104169/000119312510071652/dex13.htm','https://www.sec.gov/Archives/edgar/data/104169/000119312511083157/dex13.htm','https://www.sec.gov/Archives/edgar/data/104169/000119312512134679/d270972dex13.htm','https://www.sec.gov/Archives/edgar/data/200406/000020040613000038/ex13-form10xk20121230.htm','https://www.sec.gov/Archives/edgar/data/200406/000020040614000033/ex13-form10xk20131229.htm','https://www.sec.gov/Archives/edgar/data/104169/000010416915000011/wmtars-1312015.htm']

GEyears = [2007]
GEresLst = ['https://www.sec.gov/Archives/edgar/data/40545/000004054508000011/ex13.htm']

PGyears = [i for i in range(2007,2012)]
PGresLst = ['https://www.sec.gov/Archives/edgar/data/80424/000095015207007152/l27490aexv13.htm#303','https://www.sec.gov/Archives/edgar/data/80424/000095015208007680/l32996bexv13.htm','https://www.sec.gov/Archives/edgar/data/80424/000119312509175986/dex13.htm','https://www.sec.gov/Archives/edgar/data/80424/000119312510188769/dex13.htm','https://www.sec.gov/Archives/edgar/data/80424/000008042411000014/exhibit13.htm']

Pyears = [2008, 2012]
PresLst = ['https://www.sec.gov/Archives/edgar/data/78003/000119312509040568/dex13.htm','https://www.sec.gov/Archives/edgar/data/78003/000007800313000009/pfe-12312012xex1310ka.htm']

ATTyears = [2007, 2008, 2009, 2012]
ATTresLst = ['https://www.sec.gov/Archives/edgar/data/732717/000073271708000012/ex13.htm','https://www.sec.gov/Archives/edgar/data/732717/000073271709000007/ex13.htm','https://www.sec.gov/Archives/edgar/data/732717/000073271710000013/ex13.htm','https://www.sec.gov/Archives/edgar/data/732717/000073271712000025/ex13.htm']

In [188]:
ATTtext = pullMDA(ATTyears, 'AT&T CORP', ATTresLst)

IBMtext = pullMDA(IBMyears, 'INTL BUSINESS MACHINES CORP', IBMresLst)

# Pfizer
Ptext = pullMDA(Pyears, 'PFIZER INC', PresLst)

#Proctor and Gamble 
PGtext = pullMDA(PGyears, 'PROCTER & GAMBLE CO',PGresLst, 'The purpose of this discussion is to provide an understanding')

# GE
GEtext = pullMDA(GEyears,'GENERAL ELECTRIC CO',GEresLst)

# Wal-Mart
WMtext = pullMDA(WMyears,'WALMART INC',WMresLst)

# Johnson & Johnson
JJtext = pullMDA(JJyears,'JOHNSON & JOHNSON',JJresLst)

# Wells Fargo 
WFtext = pullMDA(WFyears,'Wells Fargo',WFresLst)

In [196]:
dd = defaultdict(list)
for d in (GEtext, WMtext,JJtext,WFtext,ATTtext,PGtext,Ptext,IBMtext): 
    for key, value in d.items():
        dd[key].append(value)

In [197]:
dfScrape = pd.DataFrame.from_dict(dd,orient='index')

In [198]:
dfScrape = dfScrape.transpose()

In [200]:
top10_MDA = pd.read_pickle('Top10_MDA.pkl')

In [201]:
frames = [top10_MDA, dfScrape]
top10Mda = pd.concat(frames)

In [202]:
top10Mda.reset_index(inplace=True)

In [170]:
top10Mda

,index,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,0,"(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL ST...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND A...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN..."
1,1,"(GENERAL ELECTRIC CO, None)","(WALMART INC, None)","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSIO...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIA...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS..."
2,2,"(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION...","(PROCTER & GAMBLE CO, None)","(WALMART INC, None)","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENT’S DISCUSSION A..."
3,3,"(AT&T CORP, None)","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSIO...","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(BERKSHIRE HATHAWAY, [ 7 MANAGEMENTS DISCUSSI...","(INTL BUSINESS MACHINES CORP, None)","(GENERAL ELECTRIC CO, [7 “MANAGEMENT’S DISCUSS...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN..."
4,4,"(PROCTER & GAMBLE CO, None)","(AT&T CORP, None)","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(GENERAL ELECTRIC CO, [7 MANAGEMENTS DISCUSS...","(CHEVRON CORP, [15. EXHIBITS, ...","(INTL BUSINESS MACHINES CORP, None)","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ..."
5,5,"(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(JOHNSON & JOHNSON, None)","(JOHNSON & JOHNSON, None)","(WALMART INC, None)","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(CHEVRON CORP, [15. EXHIBITS, FINANCIAL STATEM...","(General Electric, [7 “MANAGEMENT’S DISCUSSION...","(Johnson & Johnson, None)","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISC..."
6,6,"(CHEVRON CORP, [15. EXHIBITS, ...","(GENERAL ELECTRIC CO, [7 MANAGEMENT’S DISCUSSI...","(PROCTER & GAMBLE CO, None)","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(WALMART INC, None)","(JOHNSON & JOHNSON, None)","(Johnson & Johnson, None)","(Wells Fargo, None)","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Johnson & Johnson, [7MANAGEMENT’S DISCUSSION ..."
7,7,"(JOHNSON & JOHNSON, None)","(CHEVRON CORP, [15. EXHIBITS, ...","(INTL BUSINESS MACHINES CORP, None)","(CHEVRON CORP, [15. EXHIBITS, ...","(GENERAL ELECTRIC CO, [7 “MANAGEMENT’S DISCUSS...","(AT&T CORP, None)","(Wal-Mart Stores Inc, None)","(Wal-Mart Stores Inc, None)","(General Electric, [ 22 MANAGEMENT'S DIS...","(Johnson & Johnson, [7MANAGEMENT’S DISCUSSION ...","(JPMorgan Chase & Co., [601 OF REGULATION S-K...."
8,8,"(WA

In [204]:
top10Mda.to_pickle('topMda10.pkl')

In [158]:
for index, row in result.iterrows():
        #if result[col][1][1] == None:
        #print(col)

        result[col]
            #result.drop(result.iloc[col])
            

0     (Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...
1     (Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...
2     (Microsoft Corp., [7 MANAGEMENT’S DISCUSSION A...
3     (Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...
4     (Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...
5     (Berkshire Hathaway Inc, [7  MANAGEMENTS DISC...
6     (Johnson & Johnson, [7MANAGEMENT’S DISCUSSION ...
7     (JPMorgan Chase & Co., [601 OF REGULATION S-K....
8     (Exxon Mobil Corp., [16.     FORM 10-K SUMMARY...
9     (Bank of America Corp., [7 MANAGEMENT’S DISCUS...
10                                                  NaN
11                                                  NaN
12                                                  NaN
13                                                  NaN
14                                                  NaN
Name: 2017, dtype: object
0     (Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...
1     (Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...
2     (Microsoft Corp.

0     (Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...
1     (Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...
2     (Microsoft Corp., [7 MANAGEMENT’S DISCUSSION A...
3     (Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...
4     (Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...
5     (Berkshire Hathaway Inc, [7  MANAGEMENTS DISC...
6     (Johnson & Johnson, [7MANAGEMENT’S DISCUSSION ...
7     (JPMorgan Chase & Co., [601 OF REGULATION S-K....
8     (Exxon Mobil Corp., [16.     FORM 10-K SUMMARY...
9     (Bank of America Corp., [7 MANAGEMENT’S DISCUS...
10                                                  NaN
11                                                  NaN
12                                                  NaN
13                                                  NaN
14                                                  NaN
Name: 2017, dtype: object
0     (Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...
1     (Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...
2     (Microsoft Corp.

In [10]:
dfTop10 = pd.read_pickle("Top10_final.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'Top10_final.pkl'